In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from crewai_tools import DallETool
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")
os.environ["CREW_TRACES"] = "false"
os.environ["CREW_ASK_APPROVAL"] = "false"

In [2]:
videoScriptwriter =  Agent(
    role="Roteirista de Vídeo",
    goal="Crie um roteiro detalhado de vídeo para o Youtube voltado ao mundo dos games de acordo com o {topic}",
    backstory="""Você é especialista em criação de conteúdo e storytelling para vídeo games. Você deve criar um roteiro
    de vídeo para o roteiro utilizando o tema: {topic}.
    Você deve coletar informações sobre o {topic} para ajudar na criatividade do vídeo.
    """,
    verbose = True,
    llm = llm,
    max_iters = 5,
    allow_delegation = False,
    memory = True
)

In [3]:
class DuckDuckGoSearchTool(BaseTool):
    name: str = "duckduckgo_search"
    description: str = "Perform a DuckDuckGo news search. Input must be a search query string."


    def _run(self, query: str):
        # Proteção: tool deve receber argumento
        if not query or not isinstance(query, str):
            return {"error": "You must provide a valid query string to the tool."}

        # cria objeto de busca
        searcher = DuckDuckGoSearchResults(backend="news", num_results=10, region="br-pt")

        # DuckDuckGoSearchResults costuma ser "callable"
        try:
            results = searcher(query)
        except TypeError:
            # fallback caso a versão exija .run() ou .search()
            if hasattr(searcher, "run"):
                results = searcher.run(query)
            elif hasattr(searcher, "search"):
                results = searcher.search(query)
            else:
                return {"error": "DuckDuckGoSearchResults cannot perform searches on this version."}

        # retorno deve ser serializável
        if not results:
            return {"error": f"No news found for query '{query}'"}

        return results

    async def _arun(self, query: str):
        return self._run(query)

In [4]:
writeScript = Task(
    description="""Utilizando a ferramenta de busca, procure informações, notícias, curiosidades sobre o {topic} para embasar e ter mais criatividade para o roteiro.
    """,
    expected_output="O Roteiro do vídeo com 5 parágrafos com os principais pontos sobre o {topic}.",
    tools= [DuckDuckGoSearchTool()],
    agent = videoScriptwriter
)

In [5]:
thumbnailMaker = Agent(
    role="Criador de Thumbnail",
    goal="Produzir thumbnails utilizando o roteiro criado à partir do {topic}",
    backstory="""Você é um designer gráfico na qual sua especialidade é criação de thumbnails.
    Você cria imagens atrativas, chamativas e que despertam atenção das pessoas
    """,
    verbose = True,
    llm = llm,
    max_iters = 5,
    allow_delegation = False,
    memory = True
)

In [6]:
class ThumbnailCreator(BaseTool):
    name: str = "thumbnail_creator"
    description: str = "Creates a simple thumbnail image. Input must be a dict: {'title': '...'}"

    def _run(self, data: dict):
        if "title" not in data:
            return {"error": "Missing 'title' in data."}

        title = data["title"]

        from PIL import Image, ImageDraw
        img = Image.new("RGB", (1280, 720), "black")
        draw = ImageDraw.Draw(img)
        draw.text((50, 50), title, fill="white")

        img.save("thumbnail.png")
        return {"file": "thumbnail.png"}

In [21]:
makeThumbnail = Task(
    description="""Crie três opções diferentes de thumbnails criativas e chamativas com base no roteiro recebido. 
    Utilize a ferramenta para geração das imagens. Passa a descrição para a ferramente DallETool utilizando o contexto do histórico e o
    {topic}
    """,
    expected_output="Três thumbnails diferentes devem ser criadas a partir dos títulos gerados.",
    tools=[DallETool()],
    agent=thumbnailMaker
)


In [22]:
reviewer = Agent(
    role="Revisor",
    goal="Revisar o texto do roteiro sobre o {topic} e escolher a melhor thumbnail para o roteiro",
    backstory="""Você é um revisor e criador de conteúdo para o youtube.
    Você tem experiência com roteiros de vídeo, bem como uma boa percepção de imagens que combinem com o texto do vídeo.
    """,
    verbose = True,
    llm = llm,
    max_iters = 5,
    allow_delegation = False,
    memory = True
)

In [24]:
makeReview = Task(
    description="""Revisar o roteiro sobre o {topic} e observar as imagens de thumbnail produzidas. Escolher a imagem que melhor se adequa
    ao roteiro. O roteiro deve ser devidamente revisado, e corrigido caso tenha alguma incoerência ou não seja um texto agradável.
    """,
    expected_output="O roteiro do vídeo devidamente finalizado bem como a imagem de thumbnail escolhida",
    tools= [],
    agent = reviewer
)

In [25]:
crew = Crew(
    agents= [videoScriptwriter, thumbnailMaker, reviewer],
    tasks=[writeScript, makeThumbnail, makeReview],
    verbose=True,
    process=Process.sequential,
    share_crew=False,
    manager_llm=llm
)

In [26]:
result = crew.kickoff(inputs={'topic': 'Final Fantasy'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d09ed930-4870-4e9e-a9ac-dfb4c63c5d8c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Roteirista de Vídeo                                                                                     │
│                                                                                                                 │
│  Task: Utilizando a ferramenta de busca, procure informações, notícias, curiosidades sobre o Final Fantasy      │
│  para embasar e ter mais criatividade para o roteiro.                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/henrique/.local/share/virtualenvs/agent-ia-video-MS3pMMz9/lib/python3.13/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/henrique/.local/share/virtualenvs/agent-ia-video-MS3pMMz9/lib/python3.13/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Roteirista de Vídeo                                                                                     │
│                                                                                                                 │
│  Thought: Action: duckduckgo_search                                                                             │
│                                                                                                                 │
│  Using Tool: duckduckgo_search                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Revisor                                                                                                 │
│                                                                                                                 │
│  Task: Revisar o roteiro sobre o Final Fantasy e observar as imagens de thumbnail produzidas. Escolher a        │
│  imagem que melhor se adequa                                                                                    │
│      ao roteiro. O roteiro deve ser devidamente revisado, e corrigido caso tenha alguma incoerência ou não      │
│  seja um texto agradável.                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Revisor                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Roteiro Revisado: A Magia de Final Fantasy: Um Mergulho no Mundo dos JRPGs**                                 │
│                                                                                                                 │
│  **Introdução:**                                                                                                │
│  Se você é um fã de jogos de RPG, é quase impossível não ter ouvido falar da gigantesca franquia Final          │
│  Fantasy. Desde seu lançamento inicial em 1987, esta série revolucionou o gênero, unindo histórias              │
│  emocionantes, personagens memoráveis e uma jogabilidade inovadora. Neste vídeo, vamos explorar a rica          │
│  história de Final Fantasy, suas inovações e o impacto duradouro que teve na indústria dos games.               │
│                                                                                                                 │
│  **Parágrafo 1: A Origem da Série e sua Evolução**                                                              │
│  Final Fantasy nasceu da mente criativa de Hironobu Sakaguchi, que viu neste projeto a última chance de salvar  │
│  sua carreira na Square (hoje Square Enix). O primeiro jogo foi uma verdadeira revolução, trazendo elementos    │
│  que se tornariam marcas registradas da franquia. Com o passar dos anos, os títulos sucessores, como Final      │
│  Fantasy VII e Final Fantasy X, cimentaram seu lugar no panteão dos jogos, explorando mundos abertos e          │
│  sentimentos profundos sobre amor, perda e redenção.                                                            │
│                                                                                                                 │
│  **Parágrafo 2: O Impacto e Legado de Final Fantasy VII**                                                       │
│  Entre todos os títulos, Final Fantasy VII é frequentemente considerado o mais icônico. Lançado em 1997,        │
│  apresentou gráficos 3D impressionantes e uma narrativa que abordava temas como ecologia e identidade. O jogo   │
│  não só se tornou um fenômeno mundial, mas também ajudou a popularizar os RPGs japoneses no Ocidente. O         │
│  recente remake de Final Fantasy VII trouxe novos vislumbres para a história, expandindo o universo e           │
│  aprofundando os personagens que amamos.                                                                        │
│                                                                                                                 │
│  **Parágrafo 3: Inovações e Mecânicas de Jogo**                                                                 │
│  Final Fantasy não é apenas uma referência narrativa, mas também uma série que se destacou pela inovação em     │
│  suas mecânicas de jogo. Do sistema de combate baseado em turnos de Final Fantasy X ao revolucionário sistema   │
│  de habilidades do Final Fantasy XII, cada título trouxe algo novo. O trailer do próximo Final Fantasy XVI      │
│  sugere que a série continuará a evoluir, prometendo uma jogabilidade intensa e envolvente.                     │
│                                                                                                                 │
│  **Parágrafo 4: Os Personagens Carismáticos e Suas Histórias**                                                  │
│  Os personagens de Final Fantasy se tornaram verdadeiro

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ef0f7414-a78f-4a4f-a35e-1d3392ac0a15                                                                     │
│  Agent: Revisor                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d09ed930-4870-4e9e-a9ac-dfb4c63c5d8c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Roteiro Revisado: A Magia de Final Fantasy: Um Mergulho no Mundo dos JRPGs**                   │
│                                                                                                                 │
│  **Introdução:**                                                                                                │
│  Se você é um fã de jogos de RPG, é quase impossível não ter ouvido falar da gigantesca franquia Final          │
│  Fantasy. Desde seu lançamento inicial em 1987, esta série revolucionou o gênero, unindo histórias              │
│  emocionantes, personagens memoráveis e uma jogabilidade inovadora. Neste vídeo, vamos explorar a rica          │
│  história de Final Fantasy, suas inovações e o impacto duradouro que teve na indústria dos games.               │
│                                                                                                                 │
│  **Parágrafo 1: A Origem da Série e sua Evolução**                                                              │
│  Final Fantasy nasceu da mente criativa de Hironobu Sakaguchi, que viu neste projeto a última chance de salvar  │
│  sua carreira na Square (hoje Square Enix). O primeiro jogo foi uma verdadeira revolução, trazendo elementos    │
│  que se tornariam marcas registradas da franquia. Com o passar dos anos, os títulos sucessores, como Final      │
│  Fantasy VII e Final Fantasy X, cimentaram seu lugar no panteão dos jogos, explorando mundos abertos e          │
│  sentimentos profundos sobre amor, perda e redenção.                                                            │
│                                                                                                                 │
│  **Parágrafo 2: O Impacto e Legado de Final Fantasy VII**                                                       │
│  Entre todos os títulos, Final Fantasy VII é frequentemente considerado o mais icônico. Lançado em 1997,        │
│  apresentou gráficos 3D impressionantes e uma narrativa que abordava temas como ecologia e identidade. O jogo   │
│  não só se tornou um fenômeno mundial, mas também ajudou a popularizar os RPGs japoneses no Ocidente. O         │
│  recente remake de Final Fantasy VII trouxe novos vislumbres para a história, expandindo o universo e           │
│  aprofundando os personagens que amamos.                                                                        │
│                                                                                                                 │
│  **Parágrafo 3: Inovações e Mecânicas de Jogo**                                                                 │
│  Final Fantasy não é apenas uma referência narrativa, mas também uma série que se destacou pela inovação em     │
│  suas mecânicas de jogo. Do sistema de combate baseado em turnos de Final Fantasy X ao revolucionário sistema   │
│  de habilidades do Final Fantasy XII, cada título trouxe algo novo. O trailer do próximo Final Fantasy XVI      │
│  sugere que a série continuará a evoluir, prometendo uma jogabilidade intensa e envolvente.                     │
│                                                                                                                 │
│  **Parágrafo 4: Os Personagens Carismáticos e Suas His

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [27]:
Markdown(result.raw)

**Roteiro Revisado: A Magia de Final Fantasy: Um Mergulho no Mundo dos JRPGs**

**Introdução:**
Se você é um fã de jogos de RPG, é quase impossível não ter ouvido falar da gigantesca franquia Final Fantasy. Desde seu lançamento inicial em 1987, esta série revolucionou o gênero, unindo histórias emocionantes, personagens memoráveis e uma jogabilidade inovadora. Neste vídeo, vamos explorar a rica história de Final Fantasy, suas inovações e o impacto duradouro que teve na indústria dos games.

**Parágrafo 1: A Origem da Série e sua Evolução**
Final Fantasy nasceu da mente criativa de Hironobu Sakaguchi, que viu neste projeto a última chance de salvar sua carreira na Square (hoje Square Enix). O primeiro jogo foi uma verdadeira revolução, trazendo elementos que se tornariam marcas registradas da franquia. Com o passar dos anos, os títulos sucessores, como Final Fantasy VII e Final Fantasy X, cimentaram seu lugar no panteão dos jogos, explorando mundos abertos e sentimentos profundos sobre amor, perda e redenção.

**Parágrafo 2: O Impacto e Legado de Final Fantasy VII**
Entre todos os títulos, Final Fantasy VII é frequentemente considerado o mais icônico. Lançado em 1997, apresentou gráficos 3D impressionantes e uma narrativa que abordava temas como ecologia e identidade. O jogo não só se tornou um fenômeno mundial, mas também ajudou a popularizar os RPGs japoneses no Ocidente. O recente remake de Final Fantasy VII trouxe novos vislumbres para a história, expandindo o universo e aprofundando os personagens que amamos.

**Parágrafo 3: Inovações e Mecânicas de Jogo**
Final Fantasy não é apenas uma referência narrativa, mas também uma série que se destacou pela inovação em suas mecânicas de jogo. Do sistema de combate baseado em turnos de Final Fantasy X ao revolucionário sistema de habilidades do Final Fantasy XII, cada título trouxe algo novo. O trailer do próximo Final Fantasy XVI sugere que a série continuará a evoluir, prometendo uma jogabilidade intensa e envolvente.

**Parágrafo 4: Os Personagens Carismáticos e Suas Histórias**
Os personagens de Final Fantasy se tornaram verdadeiros ícones na cultura gamer. Desde o heroico Cloud Strife até a carismática Yuna, cada personagem tem sua própria história de crescimento e transformação. O amor, a amizade e a traição estão entrelaçados nas narrativas, criando uma conexão emocional que mantém os jogadores voltando para mais. É interessante notar que muitos dos personagens icônicos são inspirados em experiências da vida real de seus criadores.

**Conclusão:**
Final Fantasy não é apenas uma série de jogos; é um fenômeno cultural que continua a encantar e inspirar novas gerações. Com desenvolvimentos promissores, como o aguardado Final Fantasy VII Remake Parte 3, o futuro da franquia parece mais brilhante do que nunca. Então, qual é o seu título favorito da série e o que ele significa para você? Não se esqueça de deixar seu comentário e se inscrever para mais conteúdo sobre o mundo dos games!

---

**Escolha da Thumbnail:**
Após avaliar as opções disponíveis, a melhor thumbnail para este vídeo é a **Thumbnail 3:** destacando um grupo de personagens icônicos, incluindo Cloud, Tifa e Squall, em poses dinâmicas dentro de um cenário de batalha épico. Esta imagem não apenas representa a essência do que a franquia Final Fantasy é — a união de personagens memoráveis em confrontos grandiosos — mas também chamará a atenção dos espectadores, encapsulando instantaneamente a magia e a ação que os fãs esperam.

Esta combinação do roteiro revisado e da escolha da thumbnail resulta em um conteúdo envolvente e visualmente atraente, ideal para capturar a audiência do canal.